<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Classification de dépêches d’agence avec NLTK
## Nelson Jeanrenaud & Vincent Peer
## 11.06.2023

**Objectif**

L’objectif de ce labo est de réaliser des expériences de classification de documents sous NLTK avec le
corpus de dépêches Reuters. Le labo est à effectuer en binôme. Le labo sera jugé sur la qualité des
expériences et sur la discussion des différentes options explorées. Vous devez remettre un notebook
Jupyter présentant vos choix, votre code, vos résultats et les discussions. 

### Description des expériences

1. **L’objectif général** est d’explorer au moins deux aspects parmi les choix qui se posent lors de
la création d’un système probabiliste de classification de textes.



2. **Données** : les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Vous respecterez
notamment la division en données d’entraînement (train) et données de test

3. **Hyper-paramètres** : veuillez étudier au moins deux hyperparamètres. Pour chacun, veuillez
comparer au moins deux valeurs et indiquer laquelle fournit le meilleur score. Vous pourrez
choisir parmi les hyperparamètres suivants :  
• options de prétraitement des textes : stopwords, lemmatisation, tout en minuscules.  
• options de représentation : présence/absence de mots indicateurs, nombre de mots  
indicateurs ; présence/absence/nombre de bigrammes, trigrammes ; autres traits :
longueur de la dépêche, rapport tokens/types.  
• classifieurs et leurs paramètres : divers choix possibles (voir la documentation NLTK).

4. Veuillez définir et entraîner **trois classifieurs binaires** : chacun prédit si une dépêche est
étiquetée ou non avec la catégorie respective. Le premier classifieur binaire sera pour
l’étiquette ‘money-fx’, le deuxième concernera ‘grain’, et le troisième sera pour ‘nat-gas’.

5. Pour chacun des classifieurs, optimisez les hyperparamètres sans toucher aux données de test
NLTK. Divisez les données d’entraînement NLTK en 80% train et 20% dev, et choisissez les
options qui donnent les meilleurs scores sur dev.

In [12]:
import nltk
#nltk.download('reuters')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import string
import collections

from nltk.corpus import reuters
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.metrics.scores import (precision, recall, f_measure)
from random import shuffle


# Extract fileids from the reuters corpus
file_ids = reuters.fileids()
documents = []

# Loop through each file id and collect each files categories and tokenized words
for file in file_ids:
    words = reuters.words(file)
    documents.append((words, reuters.categories(file)))

shuffle(documents)
documents[0]

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vincent\AppData\Roaming\nltk_data...


(['CSCE', 'TO', 'PUT', 'ADDITIONAL', 'MARGIN', 'ON', ...], ['cocoa'])

>Pour la classification des documents, nous avons décidé d'utiliser la fréquence des mots. Nous avons donc commencé par déterminer la fréquence des mots du dataset, puis les 2000 mots les plus fréquents sont retournés.

In [14]:
def document_features(document, word_frequence):
    document_words = set(document)
    features = {}
    for word in word_frequence:
        features['contains({})'.format(word)] = (word in document_words)
    return features

def most_freq_words(documents, limit=2000):
    all_words = nltk.FreqDist(w
        for document in documents
        for w in document[0]
    )
    return list(all_words)[:limit]

def create_dataset(documents, tag, feature_extractor, **kwargs):
    if 'to_lower' in kwargs and kwargs['to_lower']:
        documents = list(map(lambda d: (list(map(str.lower, d[0])), d[1]), documents))

    if 'lemmatizer' in kwargs:
        lemmatizer = kwargs['lemmatizer']
        documents = list(map(lambda d: (list(map(lemmatizer.lemmatize, d[0])), d[1]), documents))
    
    if 'stopwords' in kwargs:
        stopwords = set(kwargs['stopwords'])
        documents = list(map(
            lambda d: (
                list(filter(lambda w: not w.lower() in stopwords and w[0].isalnum(), d[0])), 
                d[1]
            ), documents))
        
    analyzer_res = []
    if 'analyzer' in kwargs:
        analyzer_res = kwargs['analyzer'](documents)

    dataset = []
    for document in documents:
        dataset.append((feature_extractor(document[0], analyzer_res), tag in document[1]))
    
    shuffle(dataset)
    return dataset

def split_dataset(dataset):
    split_ratio = 0.6
    split_ratio2 = 0.8
    
    split = int(len(dataset) * split_ratio)
    split2 = int(len(dataset) * split_ratio2)

    return (dataset[:split], dataset[split:split2], dataset[split2:])

def best_classifier(documents, tag, dataset_creator, hyperparams):
    print('Finding best classifier for {}'.format(tag))
    print('----------')

    best = (None, 0.0)
    for hyperparam in hyperparams:
        dataset = dataset_creator(documents, tag, **hyperparam)
        train_set, test_set, dev_set = split_dataset(dataset)
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        acc = nltk.classify.accuracy(classifier, dev_set)
        
        if acc > best[1]:
            best = (classifier, acc)
        
        print('Accuracy using "{}": {:.2f}%'.format(hyperparam['title'], acc*100))
    return (best[0], test_set)

def ref_test_sets(testset, classifier):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(testset):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)

    return refsets, testsets

>Pour les hyperparamètres, nous avons choisi de supprimer les stopwords, la lemmatisation et de tout mettre en minuscule. En plus de tester chaque hyperparamètre indépendamment, nous avons aussi tenté d'appliquer plusieurs hyperparamètres en même temps, par exemple lemmatiser et supprimer les stopwords.

In [8]:
hyperparams = [
    {
        'title': 'To lower: no, Lemmatize: no, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
    },
    {
        'title': 'To lower: yes, Lemmatize: no, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
    },
    {
        'title': 'To lower: no, Lemmatize: yes, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'lemmatizer': WordNetLemmatizer(),
    },
    {
        'title': 'To lower: no, Lemmatize: no, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'stopwords': stopwords.words('english'),
    },
    {
        'title': 'To lower: yes, Lemmatize: no, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
        'stopwords': stopwords.words('english'),
    },
    {
        'title': 'To lower: no, Lemmatize: yes, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'lemmatizer': WordNetLemmatizer(),
        'stopwords': stopwords.words('english'),
    },
    {
        'title': 'To lower: yes, Lemmatize: yes, No stopwords: no',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
        'lemmatizer': WordNetLemmatizer(),
    },
    {
        'title': 'To lower: yes, Lemmatize: yes, No stopwords: yes',
        'feature_extractor': document_features,
        'analyzer': most_freq_words,
        'to_lower': True,
        'stopwords': stopwords.words('english'),
        'lemmatizer': WordNetLemmatizer(),
    },
]

### Classification des documents money-fx
>Si l'on compare chaque hyperparamètre seul, on peut voir que la suppression des stopwords offre un meilleur résultat que les autres. On remarque aussi que le fait de combiner la mise en minuscule avec les autres hyperparamètres améliore les scores. Et finalement, on remarque que combiner les trois hyperparamètres offre le meilleur score.

In [13]:
classifier_moneyfx, moneyfx_testset = best_classifier(documents, 'money-fx', create_dataset, hyperparams)

Finding best classifier for money-fx
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 87.44%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 88.65%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 87.95%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 91.75%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 92.26%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 90.73%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 88.51%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 92.68%


### Classification des documents grain
>Si l'on compare chaque hyperparamètres seul, on peut voir que cette fois la suppression des stopwords est nettement meilleure que les autres. Le fait de combiner la suppression des stopwords avec les autres hyperparamètres améliore nettement les scores. Et finalement, on remarque que combiner les trois hyperparamètres améliore le score (accuracy) du classifieur, mais qu'il n'est pas le meilleur. Ici le meilleur classifieur est celui qui combine la supression des stopwords et la mise en minuscule.

In [15]:
classifier_grain, grain_testset = best_classifier(documents, 'grain', create_dataset, hyperparams)

Finding best classifier for grain
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 88.23%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 85.50%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 86.47%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 92.17%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 92.54%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 90.69%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 87.63%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 92.17%


### Classification des documents nat-gas
>Pour ce classifieur nous nous retrouvons plus ou moins dans la même situation que le classifieur grain. C'est-à-dire que le meilleur classifieur avec un seul hyperparamètre et aussi celui qui supprime les stopwords. La combinaison des hyperparamètres augmente bien le score surout pour la combinaison stopword-minuscule qui offre le meilleur score. Cette fois, la combinaison des 3 hypermparamètres donne un résultat un peu en dessous des combinaison à deux paramètres.

In [16]:
classifier_natgas, natgas_testset = best_classifier(documents, 'nat-gas', create_dataset, hyperparams)

Finding best classifier for nat-gas
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 87.26%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 89.99%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 88.37%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 91.66%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 93.23%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 92.72%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 91.47%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 91.84%


In [17]:
moneyfx_refsets, moneyfx_testsets = ref_test_sets(moneyfx_testset, classifier_moneyfx)
grain_refsets, grain_testsets = ref_test_sets(grain_testset, classifier_grain)
natgas_refsets, natgas_testsets = ref_test_sets(natgas_testset, classifier_natgas)

6. Veuillez donner les scores de rappel, précision et f-mesure de chacun des trois classifieurs,
avec les meilleurs hyperparamètres, sur les données de test.


In [20]:
print('Money-fx:')
print('---------')
print('Precision:', precision(moneyfx_refsets[True], moneyfx_testsets[True]))
print('Recall:'   , recall(moneyfx_refsets[True], moneyfx_testsets[True]))
print('F-mesure:' , f_measure(moneyfx_refsets[True], moneyfx_testsets[True]))

print()

print('Grain:')
print('---------')
print('Precision:', precision(grain_refsets[True], grain_testsets[True]))
print('Recall:'   , recall(grain_refsets[True], grain_testsets[True]))
print('F-mesure:' , f_measure(grain_refsets[True], grain_testsets[True]))

print()

print('Nat-gas:')
print('---------')
print('Precision:', precision(natgas_refsets[True], natgas_testsets[True]))
print('Recall:'   , recall(natgas_refsets[True], natgas_testsets[True]))
print('F-mesure:' , f_measure(natgas_refsets[True], natgas_testsets[True]))

Money-fx:
---------
Precision: 0.38596491228070173
Recall: 0.7801418439716312
F-mesure: 0.5164319248826291

Grain:
---------
Precision: 0.3475177304964539
Recall: 0.9333333333333333
F-mesure: 0.5064599483204134

Nat-gas:
---------
Precision: 0.0663265306122449
Recall: 0.7222222222222222
F-mesure: 0.12149532710280375


En regardant la précision, le rappel et la F-Mesure de tout nos classifieurs, on remarque qu'ils ont un problème de précision. C'est-à-dire qu'ils n'arrivent pas à correctement classifier les documents du type que l'on souhaite classifier. Pour le rappel, on voit que les résultats sont corrects mais pas excellent, ce qui veut dire qu'ils sont tout de même globalement capables de classifier les documents qui ne sont pas ceux que l'on souhaite classifier. Le F-Mesure indique que les deux premiers modèles sont aux alentours de 50% ce qui n'est pas non plus une grande performance. Pire encore, le nat-gas a une f-mesure à seuelement 12%.

7. Veuillez définir **un quatrième classifieur multi-classe** qui assigne une étiquette parmi quatre :
les trois choisies ci-dessus plus la catégorie ‘other’. Vous devrez nettoyer les données, car un
petit nombre de dépêches sont annotées avec plusieurs étiquettes : dans ce cas, gardez
seulement la première. 

In [21]:
def create_multi_dataset(documents, tags, feature_extractor, **kwargs):
    if 'to_lower' in kwargs and kwargs['to_lower']:
        documents = list(map(lambda d: (list(map(str.lower, d[0])), d[1]), documents))

    if 'lemmatizer' in kwargs:
        lemmatizer = kwargs['lemmatizer']
        documents = list(map(lambda d: (list(map(lemmatizer.lemmatize, d[0])), d[1]), documents))
    
    if 'stopwords' in kwargs:
        stopwords = set(kwargs['stopwords'])
        documents = list(map(
            lambda d: (
                list(filter(lambda w: not w.lower() in stopwords and w[0].isalnum(), d[0])), 
                d[1]
            ), documents))
        
    analyzer_res = []
    if 'analyzer' in kwargs:
        analyzer_res = kwargs['analyzer'](documents)

    dataset = []
    for document in documents:
        document_tags = list(set(tags).intersection(document[1]))
        tag = 'other' if document_tags == [] else document_tags[0]

        dataset.append((feature_extractor(document[0], analyzer_res), tag))
    
    shuffle(dataset)
    return dataset

def best_multi_classifier(documents, tag, hyperparams):
    print('Finding best milti-classifier for {}'.format(tag))
    print('----------')

    best = (None, 0.0)
    for hyperparam in hyperparams:
        dataset = create_multi_dataset(documents, tag, **hyperparam)
        train_set, test_set, dev_set = split_dataset(dataset)
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        acc = nltk.classify.accuracy(classifier, dev_set)
        
        if acc > best[1]:
            best = (classifier, acc)
        
        print('Accuracy using "{}": {:.2f}%'.format(hyperparam['title'], acc*100))
    return (best[0], test_set)

In [22]:
classifier_multi, multi_testset = best_multi_classifier(documents, ['money-fx', 'grain', 'nat-gas'], hyperparams)

Finding best milti-classifier for ['money-fx', 'grain', 'nat-gas']
----------
Accuracy using "To lower: no, Lemmatize: no, No stopwords: no": 78.64%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: no": 77.34%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: no": 78.22%
Accuracy using "To lower: no, Lemmatize: no, No stopwords: yes": 82.07%
Accuracy using "To lower: yes, Lemmatize: no, No stopwords: yes": 83.13%
Accuracy using "To lower: no, Lemmatize: yes, No stopwords: yes": 82.34%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: no": 78.08%
Accuracy using "To lower: yes, Lemmatize: yes, No stopwords: yes": 83.04%


In [23]:
multi_refsets, multi_testsets = ref_test_sets(multi_testset, classifier_multi)

8. Veuillez donner les scores de rappel, précision et f-mesure de ce classifieur pour chacune des
trois étiquettes choisies. Comment les scores se comparent-ils à ceux des trois classifieurs
binaires ? 

In [24]:
words = ['money-fx', 'grain', 'nat-gas']
print("Score for multiclass classifiers for {}".format(", ".join(map(lambda i: i.title(), words))))
words.append('other')
for word in words:
    print("")
    print('{}:'.format(word.title()))
    print('---------')
    print('Precision:', precision(multi_refsets[word], multi_testsets[word]))
    print('Recall:'   , recall(multi_refsets[word], multi_testsets[word]))
    print('F-mesure:' , f_measure(multi_refsets[word], multi_testsets[word]))

Score for multiclass classifiers for Money-Fx, Grain, Nat-Gas

Money-Fx:
---------
Precision: 0.4678111587982833
Recall: 0.8195488721804511
F-mesure: 0.5956284153005464

Grain:
---------
Precision: 0.4713114754098361
Recall: 0.9274193548387096
F-mesure: 0.625

Nat-Gas:
---------
Precision: 0.15384615384615385
Recall: 0.75
F-mesure: 0.25531914893617025

Other:
---------
Precision: 0.9757033248081841
Recall: 0.8129994672349494
F-mesure: 0.8869514675966289


>Pour le multiclass, la précision est à nouveau faible mais le rappel est lui très bon. La f-mesure suit les résultats précédents en maintenant un score médiocre.

9. **Documentation** : livre NLTK, [chapitre 2](https://www.nltk.org/book/ch02.html) pour accéder au corpus Reuters et le [chapitre 6](https://www.nltk.org/book/ch06.html) pour
la classification ; puis http://www.nltk.org/howto/classify.html pour les classifieurs dans
NLTK ; enfin, Introduction to Information Retrieval (https://nlp.stanford.edu/IRbook/information-retrieval-book.html), [chapitre 13](https://nlp.stanford.edu/IR-book/html/htmledition/text-classification-and-naive-bayes-1.html), pour une discussion générale de
méthodes de classification, et des exemples de scores obtenus sur certaines étiquettes.